In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import ConcatDataset
from torch.optim import lr_scheduler

from sklearn.utils import class_weight
from sklearn.model_selection import KFold

from tqdm import tqdm

import wandb

from math import log10, floor
import random
from datetime import datetime

from process_input.input_to_2Dmatrix import parse_matrices, FeatureDataset
from model_skeletons.rnn.rnn_v1_6 import rnn, model_version
from compute_accuracy import compute_accuracy
from class_weights_cv import get_class_weights

norm=True
model_script=os.path.join("model_skeletons", "rnn", "{}.py".format(model_version)) #Used to read the file for logs
assert os.path.isfile(model_script), "Could not find file '{}'".format(model_script)
new_files_creation="create_new_data_files.py" #Used to read the file for logs
input_to_matrix=os.path.join("process_input","input_to_2Dmatrix.py") #Used to read the file for logs
assert os.path.isfile(input_to_matrix), "Could not find file '{}'".format(input_to_matrix)
compute_accuracy_file = "compute_accuracy.py"
assert os.path.isfile(compute_accuracy_file), "Could not find file '{}'".format(compute_accuracy_file)
valid_batch_size=5
log_file=os.path.join("logs", "rnn", "{}.log".format(model_version))
cv_log_file=os.path.join("logs", "rnn", "cv_{}.log".format(model_version))
last_run_log_file=os.path.join("logs", "rnn", "current_run_log_{}.log".format(model_version))
last_run_log_file_final_runs=os.path.join("logs", "rnn", "{}_all_runs.log".format(model_version))
model_path=os.path.join("saved_models", "rnn", "{}.pth".format(model_version))
cv=False
train=False


In [ ]:
#device = torch.device('cpu')
device = torch.device('cuda')
many_classes=True #ACGT order in onehot

In [ ]:
def write_to_log_file(learning_rate, optimizer_type, weight_decay, momentum, is_scheduler, T_max, smoothing, 
                      train_batch_size, valid_batch_size, log_file, model_version, 
                      input_to_matrix, new_greatest_valid_acc, model_path, many_classes,
                      compute_accuracy_file, model_script, run_name, dt_string, bidirectional, hidden_dim, n_layers,
                      norm, acc_list=None):
    """Called, when new highest validation accuracy is found.
    
    Writes everything important information to a log file."""
    with open(log_file, 'w+') as fw:
        fw.write("Created: {}\nModel version: {}\nPath: {}\nRun name: {}\nAccuracy: {}\n\n".format(dt_string, model_version, 
                                                                                                   model_path, run_name, 
                                                                                                   new_greatest_valid_acc))
        if acc_list:
            for k, accuracy in enumerate(acc_list):
                fw.write("Fold {}: {}.\n".format(k+1, accuracy))
            fw.write('\n')
        fw.write("Hyperparameters:\nOptimizer: {}\n".format(optimizer_type))
        if optimizer_type=="SGD":
            fw.write("Momentum: {}\n".format(momentum))
        fw.write("Learning rate: {}\nWeight decay: {}\n".format(learning_rate, weight_decay))
        if is_scheduler:
            fw.write("Used CosineAnnealingLR scheduler with T_max {}\n".format(T_max))
        if many_classes:
            fw.write("Used CrossEntropyLoss with label smoothing {}\n".format(smoothing))
            fw.write("Balanced classes\n")
        else:
            fw.write("Used BCELoss without label smoothing\n")
        fw.write("Data normalized: {}\n".format(norm))
        fw.write("Bidirectional: {}\n".format(bidirectional))
        fw.write("Hidden_dim: {}\n".format(hidden_dim))
        fw.write("N_layers: {}\n".format(n_layers))
        fw.write("Train batch size: {}\nValidation batch size: {}\n\n".format(train_batch_size, valid_batch_size))
        
        fw.write("\n--------------------------Script of the model can be seen below.---------------------------\n")
        with open(model_script, 'r') as fr:
            fw.write(fr.read())
        fw.write("\n-------------------------------------------------------------------------------------------")
        fw.write("\n\n\n")
    
        fw.write("\n------------------------------Created input matrices with script:------------------------------\n")
        with open(input_to_matrix, 'r') as fr:
            fw.write(fr.read())
        fw.write("\n-------------------------------------------------------------------------------------------")
        fw.write("\n\n\n")
        
        fw.write("\n------------------------------Computed accuracies with script:-----------------------------\n")
        with open(compute_accuracy_file, 'r') as fr:
            fw.write(fr.read())
        fw.write("\n-------------------------------------------------------------------------------------------\n")

In [ ]:
"""In input file, 33 base, 17th is altering, 1st A, 2nd C, 3rd G, 4th T"""
class_types=['AA', 'AC', 'AG', 'AT', 'CA', 'CC', 'CG', 'CT', 'GA', 'GC', 'GG',
            'GT', 'TA', 'TC', 'TG', 'TT']
classes={'CA': 0, 'CC': 1, 'CG': 2, 'CT': 3, 'TA': 4, 'TC': 5, 'TG': 6, 'TT': 7}

In [ ]:
train_file="data/20220214/sompred_crc9_clu1_pyri_mut_combined_train.matrix"
valid_file="data/20220214/sompred_crc9_clu1_pyri_mut_combined_valid.matrix"
test_file="data/20220214/sompred_crc9_clu1_pyri_mut_combined_test.matrix"

In [ ]:
trainset_input, trainset_target = parse_matrices(train_file, classes, norm)
valid_input, valid_target = parse_matrices(valid_file, classes, norm)
test_input, test_target = parse_matrices(test_file, classes, norm)

In [ ]:
train_dataset=FeatureDataset(data=trainset_input, labels=trainset_target)
valid_dataset=FeatureDataset(data=valid_input, labels=valid_target)
test_dataset=FeatureDataset(data=test_input, labels=test_target)
combined_valid_train = ConcatDataset([train_dataset, valid_dataset]) #Combines validation and training datasets for cross validation

In [ ]:
def cv_train_network(net,criterion, valid_criterion,epochs,optimizer,trainloader, validloader,
                  is_scheduler,scheduler, wandb, early_stop=100):
    j=0
    greatest_acc=0
    min_tot_loss=float('inf')
    tot_loss=0
    tot_items=0
    for i in range(epochs):
        net.train()
        for sequences, labels in trainloader:
            sequences, labels = sequences.to(device), labels.to(device)
            optimizer.zero_grad()
            out, hidden = net(sequences)
            out = out.squeeze()
            loss=criterion(out,labels)
            tot_loss+=loss.item()
            tot_items+=len(labels)
            loss.backward()
            if torch.isnan(loss):
                raise RuntimeError("NAN!")
            optimizer.step()
        if is_scheduler:
            scheduler.step()
        tot_loss/=tot_items
        accuracy, tot_valid_loss = compute_accuracy(device, net, validloader, valid_criterion, 
                                                "VALID", verbose = False, cv=True)
        if wandb!=None:
            wandb.log({"Training loss": tot_loss,
                       "Validation loss": tot_valid_loss,
                       "Valid Accuracy": accuracy,
            #           "Test loss": test_loss,
            #           "Test Accuracy": test_accuracy,
            #           "Pooled test recall": fake_recall_test,
            #           "Pooled test precision": fake_precision_test,
            #           "Learning rate": optimizer.param_groups[0]['lr'],
            #           "Scheduler": is_scheduler,
                       "Epoch": i})
        if round(accuracy,3)<=round(greatest_acc,3):
            pass
        else:
            greatest_acc=accuracy
        if round(tot_valid_loss,3)>=round(min_tot_loss,3):
            j+=1
            if j>=early_stop and i>100:
                break
        else:
            j=0
            min_tot_loss=tot_valid_loss
        
    return greatest_acc

In [ ]:
def train_network(net,criterion, valid_criterion,epochs,optimizer,trainloader, validloader,
                  is_scheduler,scheduler, greatest_acc_overall, model_path, wandb, early_stop=50):
    j=0
    greatest_acc=0
    tot_loss=0
    tot_items=0
    for i in range(epochs):
        net.train()
        for sequences, labels in trainloader:
            sequences, labels = sequences.to(device), labels.to(device)
            optimizer.zero_grad()
            out, hidden = net(sequences)
            out=out.squeeze()
            loss=criterion(out,labels)
            tot_loss+=loss.item()
            tot_items+=len(labels)
            loss.backward()
            if torch.isnan(loss):
                raise RuntimeError("NAN!")
            optimizer.step()
        if is_scheduler:
            scheduler.step()
        tot_loss/=tot_items
        accuracy, tot_valid_loss = compute_accuracy(device, net, validloader, valid_criterion, "VALID", verbose = False, cv = True)
        
        if wandb!=None:
            wandb.log({"Training loss": tot_loss,
                       "Validation loss": tot_valid_loss,
                       "Valid Accuracy": accuracy,
            #           "Test loss": test_loss,
            #           "Test Accuracy": test_accuracy,
            #           "Pooled test recall": fake_recall_test,
            #           "Pooled test precision": fake_precision_test,
            #           "Learning rate": optimizer.param_groups[0]['lr'],
            #           "Scheduler": is_scheduler,
                       "Epoch": i})
            
        if round(accuracy,3)<=round(greatest_acc,3):
            if early_stop:
                j+=1
                if j>=early_stop and i>100:
                    print("Greates validation acc: {}".format(greatest_acc))
                    break
        else:
            if accuracy>greatest_acc_overall:
                torch.save(net.state_dict(), model_path)
                greatest_acc_overall=accuracy
            j=0
            greatest_acc=accuracy
    print("Greatest accuracy on run: {}".format(greatest_acc))
    return greatest_acc

In [ ]:
def get_earlier_accuracy(log_file):
    with open(log_file, 'r') as fr:
        for line in fr:
            if "Accuracy:" in line:
                return float(line.strip().split(' ')[1]) #Accuracy is written as Accuracy: <acc>

In [ ]:
greatest_avg_valid_acc = 0
if os.path.isfile(cv_log_file):
    greatest_avg_valid_acc = get_earlier_accuracy(cv_log_file)
print(greatest_avg_valid_acc)

In [ ]:
if not os.path.isfile(last_run_log_file):
    with open(last_run_log_file, 'w+') as fw:
        fw.write("Run log.\n\n")

In [ ]:
#k_fold cross_validation for hyperparameters
if cv:
    k_folds = 5
    epochs=5000
    run_name="None"
    testloader=None

    kfold = KFold(n_splits=k_folds, shuffle=True) #batch size affects the size of datasets
    for i in range(150): #Test with 30 different hyperparameter combinations
        bidirectional=random.sample([True,False], 1)[0]
        hidden_dim=random.sample(range(1,100), 1)[0]
        n_layers=random.sample(range(1,5), 1)[0]
        
        valid_accuracies = list()
        mom_text=""
        scheduler_text=""
        momentum=None
        T_max=None
        is_scheduler=False
        scheduler=None
        learning_rate=random.sample([0.01, 0.001, 0.0001, 0.00001], 1)[0]
        lr_text=str(learning_rate).replace(".","d")
        train_batch_size=random.sample([32, 64, 128], 1)[0]

        optimizer_type=random.sample(["Adam","SGD"], 1)[0]
        if optimizer_type=="SGD":
            momentum= random.sample([0, np.random.uniform()], 1)[0]
            if momentum!=0: momentum = round(momentum, -int(floor(log10(momentum))) + 2)
            mom_text="_mom"+str(round(momentum,2)).replace(".","d")
            is_scheduler=random.sample([True, False], 1)[0]
            if is_scheduler:
                T_max=random.sample([1, np.random.uniform(low=0.2)], 1)[0]
                T_max = round(T_max, -int(floor(log10(T_max))) + 2)
        weight_decay=random.sample([0, 0.00001, 0.0001, 0.001], 1)[0]
        if weight_decay!=0: weight_decay = round(weight_decay, -int(floor(log10(weight_decay))) + 2)
        decay_text="_wdecay"+str(weight_decay).replace(".","d")
        smoothing=random.sample([0, np.random.uniform(high=0.05)], 1)[0]
        if smoothing!=0: smoothing = round(smoothing, -int(floor(log10(smoothing))) + 2)

        for (train_ids, test_ids) in kfold.split(combined_valid_train):
            # Sample elements randomly from a given list of ids, no replacement.
            train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
            test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
            trainloader = torch.utils.data.DataLoader(
                          combined_valid_train, 
                          batch_size=train_batch_size, sampler=train_subsampler)
            validloader = torch.utils.data.DataLoader(
                              combined_valid_train,
                              batch_size=valid_batch_size, sampler=test_subsampler)
            net = rnn(bidirectional, hidden_dim, n_layers).to(device)
            if optimizer_type=="Adam":
                optimizer=torch.optim.Adam(net.parameters(), lr=learning_rate, weight_decay=weight_decay)
            elif optimizer_type=="SGD":
                optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, 
                                            momentum=momentum, weight_decay=weight_decay)
                if is_scheduler:
                    scheduler=lr_scheduler.CosineAnnealingLR(optimizer, T_max=T_max*epochs)
            else:
                raise RuntimeError("WRONG OPTIMIZER: {}".format(optimizer_type))


            train_class_weights, valid_class_weights = get_class_weights(device, trainloader, validloader)
            criterion=nn.CrossEntropyLoss(weight=train_class_weights, label_smoothing=smoothing)
            valid_criterion=nn.CrossEntropyLoss(weight=valid_class_weights)
            
            os.system("rm -rf masters_thesis/wandb/run-*")
            run = wandb.init(project='rnn')
            run_name = "batch"+str(train_batch_size)+"_lr-"+lr_text \
            +"smooth"+str(round(smoothing,2)).replace('.','d')+"optim-"+optimizer_type+decay_text \
            +"_sch-"+str(is_scheduler)+scheduler_text+mom_text
            wandb.run.name = run_name
            config = wandb.config
            config.batch_size=train_batch_size
            #config.optimizer_type=optimizer_type
            #config.learning_rate = learning_rate
            #config.weight_decay=weight_decay
            #config.is_scheduler=is_scheduler

            valid_acc = cv_train_network(net,criterion,valid_criterion,epochs,optimizer,trainloader, validloader, 
                                      is_scheduler,scheduler, wandb)
            valid_accuracies.append(valid_acc)
        avg_valid_acc = sum(valid_accuracies) / len(valid_accuracies)

        print("Optimizer: {}\nLearning Rate: {}\nScheduler: {}".format(optimizer_type, learning_rate, is_scheduler))
        print("Weight decay: {}\nSmoothing: {}".format(weight_decay, smoothing))
        if optimizer_type=="SGD":
            print("Momentum: {}".format(momentum))
            if is_scheduler:
                print("T_max: {}\n".format(T_max))
        print("Average validation accuracy: {}".format(avg_valid_acc))
        for k, accuracy in enumerate(valid_accuracies):
            print("Fold {}: {}.".format(k+1, accuracy))

        with open(last_run_log_file, 'a+') as fw:
            fw.write("Time: {}\n".format(datetime.now().strftime("%d.%m.%Y %H:%M:%S")))
            fw.write("Average validation accuracy: {}\n".format(avg_valid_acc))
            fw.write("\n".join(["Fold {}: {}.".format(k+1, accuracy) for k, accuracy in enumerate(valid_accuracies)]))
            fw.write("\nOptimizer: {}\nLearning Rate: {}\nScheduler: {}\n".format(optimizer_type, learning_rate, is_scheduler))
            fw.write("Weight decay: {}\nSmoothing: {}\nBatch size: {}\n".format(weight_decay, smoothing, train_batch_size))
            fw.write("Bidirectional: {}\nHidden Dim: {}\nn_layers: {}\n".format(bidirectional, hidden_dim, n_layers))
            if optimizer_type=="SGD":
                fw.write("Momentum: {}\n".format(momentum))
                if is_scheduler:
                    fw.write("T_max: {}\n".format(T_max))
            fw.write('\n\n')

        if avg_valid_acc>greatest_avg_valid_acc:
            now = datetime.now()
            dt_string = now.strftime("%d.%m.%Y %H:%M:%S")
            write_to_log_file(learning_rate, optimizer_type, weight_decay, momentum, is_scheduler, T_max,
                             smoothing, train_batch_size, valid_batch_size, cv_log_file, model_version, 
                              input_to_matrix, 
                              avg_valid_acc, model_path, many_classes, compute_accuracy_file,
                              model_script, run_name, dt_string, bidirectional, hidden_dim, n_layers,
                              acc_list=valid_accuracies, norm=norm)
            greatest_avg_valid_acc=avg_valid_acc

In [ ]:
greatest_acc_overall=0
if os.path.isfile(log_file):
    greatest_acc_overall=get_earlier_accuracy(log_file)
print(greatest_acc_overall)
train_batch_size=32

In [ ]:
trainloader = torch.utils.data.DataLoader(train_dataset
    ,batch_size=train_batch_size
    ,shuffle=True
    ,drop_last=True
)
validloader = torch.utils.data.DataLoader(valid_dataset
    ,batch_size=5
    ,shuffle=False
)

In [ ]:
if many_classes:
    class_weights=class_weight.compute_class_weight(
        class_weight='balanced',
        classes=np.unique(trainset_target),
        y=np.array(trainset_target))
    class_weights=torch.tensor(class_weights,dtype=torch.float)
    class_weights=class_weights.to(device)
    
    valid_class_weights=class_weight.compute_class_weight(
        class_weight='balanced',
        classes=np.unique(valid_target),
        y=np.array(valid_target))
    valid_class_weights=torch.tensor(valid_class_weights,dtype=torch.float)
    valid_class_weights=valid_class_weights.to(device)
    
else:
    print(np.array(trainset_target))
print(class_weights, valid_class_weights)

In [ ]:
if train:
    learning_rate=0.0001
    momentum=None
    T_max=None
    is_scheduler=False
    optimizer_type="Adam"
    weight_decay=0.001
    smoothing=0.0432
    scheduler=None
    epochs=5000
    criterion=nn.CrossEntropyLoss(weight=class_weights, label_smoothing=smoothing)
    valid_criterion=nn.CrossEntropyLoss(weight=valid_class_weights)
    lr_text=str(learning_rate).replace(".","d")
    decay_text="_wdecay"+str(weight_decay).replace(".","d")
    mom_text=""
    scheduler_text=""
    for i in range(1000):
        net = rnn(bidirectional, hidden_dim, n_layers).to(device)
        if optimizer_type=="Adam":
            optimizer=torch.optim.Adam(net.parameters(), lr=learning_rate, weight_decay=weight_decay)
        elif optimizer_type=="SGD":
            optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, 
                                        momentum=momentum, weight_decay=weight_decay)
            mom_text="_mom"+str(round(momentum,2)).replace(".","d")
        else:
            raise RuntimeError("WRONG OPTIMIZER.")
        if is_scheduler:
            scheduler=lr_scheduler.CosineAnnealingLR(optimizer, T_max=T_max*epochs)
            scheduler_text="-cos"+str(T_max).replace(".","d")
        run = wandb.init(project='rnn_final')
        run_name="run_{}".format(i)
        wandb.run.name = run_name
        config = wandb.config
        config.batch_size=train_batch_size
        config.optimizer_type=optimizer_type
        config.learning_rate = learning_rate
        config.weight_decay=weight_decay
        config.is_scheduler=is_scheduler
        if is_scheduler:
            config.T_max=T_max
            config.scheduler="Cosine"
        if optimizer_type=="SGD":
            config.momentum = momentum

        greatest_acc = train_network(net,criterion, valid_criterion,epochs,optimizer,trainloader, validloader,
                          is_scheduler,scheduler, greatest_acc_overall, model_path, wandb, early_stop=None)

        with open(last_run_log_file_final_runs, 'a+') as fw:
                fw.write("Time: {}\n".format(datetime.now().strftime("%d.%m.%Y %H:%M:%S")))
                fw.write("Accuracy: {}\n".format(greatest_acc))
                fw.write("\nOptimizer: {}\nLearning Rate: {}\nScheduler: {}\n".format(optimizer_type, learning_rate, is_scheduler))
                fw.write("Weight decay: {}\nSmoothing: {}\nBatch size: {}\n".format(weight_decay, smoothing, train_batch_size))
                fw.write("Bidirectional: {}\nHidden Dim: {}\nn_layers: {}\n".format(bidirectional, hidden_dim, n_layers))
                if optimizer_type=="SGD":
                    fw.write("Momentum: {}\n".format(momentum))
                    if is_scheduler:
                        fw.write("T_max: {}\n".format(T_max))
                fw.write('\n\n')


        if greatest_acc>greatest_acc_overall:
            now = datetime.now()
            dt_string = now.strftime("%d.%m.%Y %H:%M:%S")
            write_to_log_file(learning_rate, optimizer_type, weight_decay, momentum, is_scheduler, T_max,
                             smoothing, train_batch_size, valid_batch_size, log_file, model_version, 
                              input_to_matrix, 
                              greatest_acc, model_path, many_classes, compute_accuracy_file,
                              model_script, run_name, bidirectional, hidden_dim, 
                              n_layers, dt_string, norm=norm)
            greatest_acc_overall=greatest_acc
                #raise Exception("LOL")
                        #run.finish()
    #learning_rate=0.001
    #momentum=0.9
    #gamma=2
    #is_scheduler=True
    #optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum)#, weight_decay=0.1-0.0001
    #scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=gamma)
    #scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=250)

In [ ]:
testloader = torch.utils.data.DataLoader(test_dataset
    ,batch_size=5
    ,shuffle=False
)

In [ ]:

#model = model_skeleton(many_classes)
model=rnn(bidirectional=False, hidden_dim=22, n_layers=1)
#model.load_state_dict(torch.load(model_path))
model.load_state_dict(torch.load(model_path))
model.eval()
model.to(device)

In [ ]:
criterion=nn.CrossEntropyLoss(weight=class_weights)
accuracy, loss = \
compute_accuracy(device, model, testloader, None, "TEST", verbose=True, cv=False)

print("Accuracy for the test data:",accuracy)
#print("Loss for test data:",loss)
# print("F1-score:",f1)
# print("F2-score:",f2)
# print("Precision:",precision)
# print("Recall:",recall)

In [ ]:
valid_criterion=nn.CrossEntropyLoss(weight=valid_class_weights)
accuracy, tot_valid_loss = compute_accuracy(device, model, validloader, valid_criterion, 
                                                "VALID", verbose = False, cv=True)
print(accuracy)

In [ ]:
class model_skeleton(nn.Module):
    def __init__(self):
        super(model_skeleton, self).__init__()
        # YOUR CODE HERE
        self.dropout=nn.Dropout(0.1)
        self.m = nn.MaxPool1d(2, stride=2)
        #self.conv1 = nn.Conv1d(1, 6, (4,1))
        self.conv1 = nn.Conv1d(1, 6, (4,1))
        self.conv2 = nn.Conv1d(6, 12, 3)
        self.conv3 = nn.Conv1d(12, 15, 3)
        self.conv4 = nn.Conv1d(10, 10, 5)
        self.conv1_2 = nn.Conv1d(1, 15, (4,33))
        self.fc1=nn.Linear(450,200)#150) 
        self.fc2=nn.Linear(200,120)
        self.fc3=nn.Linear(120,8)
        self.fc4=nn.Linear(120,1)
        self.sigmoid=nn.Sigmoid()
        

    def forward(self, x):
        """
        Args:
          x of shape (batch_size, 1, 28, 28): Input images.
        
        Returns:
          y of shape (batch_size, 10): Outputs of the network.
        """
        y=F.relu(self.conv1(x))
        y=y.squeeze(dim=2)
        #y=self.m(y)
        y=F.relu(self.conv2(y))
        #y=self.m(y)
        y=self.dropout(y)
        y=F.relu(self.conv3(y))
        #y=self.m(y)
        y=self.dropout(y)
        #y=F.relu(self.conv4(y))
        #self.dropout(y)
        y2=F.relu(self.conv1_2(x)).squeeze(dim=2)
        y=torch.cat((y,y2), dim=2)
        y=self.dropout(y)
        y = y.view(-1, self.num_flat_features(y))
        y=F.relu(self.fc1(y))
        y=self.dropout(y)
        y=F.relu(self.fc2(y))
        y=self.dropout(y)
        if self.many_classes:
            y=self.fc3(y)
        #else:
        #    y=self.fc4(y)
        #    y=self.sigmoid(y)
        return y
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [ ]:
def countProbabilityDistributions(net, dataloader):
    distributions=dict()
    x=None
    m = nn.Softmax(dim=1)
    predictions=None
    with torch.no_grad():
        tot_loss=0
        for sequences, labels in dataloader:
            sequences, labels = sequences.to(device), labels.to(device)
            outputs = net(sequences)
            _, predicted = torch.max(outputs.data, 1)
            if predictions==None:
                predictions=predicted
                trueLabels=labels
            else:
                predictions=torch.cat((predictions, predicted), 0)
                trueLabels=torch.cat((trueLabels, labels), 0)
            for i in range(outputs.data.shape[1]):
                if not i in distributions:
                    distributions[i]=m(outputs).data[:,i]
                else:
                    distributions[i]=torch.cat((distributions[i], m(outputs).data[:,i]), 0)
                    #print(m(outputs).data[0,:])
                    #print(predicted)
                    #raise RuntimeError("LOL")
    return distributions, predictions, trueLabels
#distributions, predictions, trueLabels= countProbabilityDistributions(net, trainloader)
distributions, predictions, trueLabels= countProbabilityDistributions(net, trainloader)

In [ ]:
print(classes)
plt.hist(predictions.to("cpu").numpy(),bins=[0,1,2,3,4,5,6,7,8,9]);

In [ ]:
trueLabels[3]

In [ ]:
classes

In [ ]:
classNames=['CA', 'CC', 'CG', 'CT', 'TA', 'TC', 'TG', 'TT']
for i in range(8):
    plt.figure(figsize=(20,20))
    condition=(predictions==i) & (trueLabels==i)
    plt.hist(distributions[i][condition].to("cpu").numpy(),bins=100, color='r', alpha=0.5);#, density=True);
    condition=(predictions==i) & (trueLabels!=i)
    plt.hist(distributions[i][condition].to("cpu").numpy(),bins=100, color='b', alpha=0.5);#, density=True);
    plt.title(classNames[i]);
    plt.xlim([0,1])
    plt.savefig('probDistr_{}.pdf'.format(classNames[i]))

In [ ]:
print(classes)
plt.figure(figsize=(20,20))
condition=(predictions==1)
plt.hist(distributions[1][condition].to("cpu").numpy(),bins=100, color='r', alpha=0.5, density=True);
other=torch.cat((distributions[0][predictions==0],distributions[2][predictions==2],distributions[3][predictions==3]),dim=0)
plt.hist(other.to("cpu").numpy(),bins=100, color='b', alpha=0.5, density=True);

In [ ]:
fig,ax = plt.subplots(2,2,constrained_layout = True) # Instantiate figure and axes object
ax[0][0].hist(distributions[0][predictions==0].to("cpu").numpy(),bins=100, color='r', density=True);
ax[0][0].title.set_text('CA 475')
ax[0][1].hist(distributions[1][predictions==1].to("cpu").numpy(),bins=100, color='r', density=True);
ax[0][1].title.set_text('CC 3048')
ax[1][0].hist(distributions[2][predictions==2].to("cpu").numpy(),bins=100, color='r', density=True);
ax[1][0].title.set_text('CG 127')
ax[1][1].hist(distributions[3][predictions==3].to("cpu").numpy(),bins=100, color='r', density=True);
ax[1][1].title.set_text('CT 639')

In [ ]:
fig,ax = plt.subplots(2,2,constrained_layout = True) # Instantiate figure and axes object
ax[0][0].hist(distributions[4][predictions==4].to("cpu").numpy(),bins=100, color='r', density=True);
ax[0][0].title.set_text('TA 181')
ax[0][1].hist(distributions[5][predictions==5].to("cpu").numpy(),bins=100, color='r', density=True);
ax[0][1].title.set_text('TC 303')
ax[1][0].hist(distributions[6][predictions==6].to("cpu").numpy(),bins=100, color='r', density=True);
ax[1][0].title.set_text('TG 122')
ax[1][1].hist(distributions[7][predictions==7].to("cpu").numpy(),bins=100, color='r', density=True);
ax[1][1].title.set_text('TT 4285')

In [ ]:
fig,ax = plt.subplots(2,4) # Instantiate figure and axes object
for index in range(8):
    ax[int(index>3)][index-(index>3)*4].hist(distributions[index].to("cpu").numpy(), density=True, bins=20)#,
            #bins=[0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9], density=True, histtype="step")
    ax[int(index>3)][index-(index>3)*4].title.set_text(str(index))
plt.legend()
plt.show()

In [ ]:
fig,axes = plt.subplots(2,1) # Instantiate figure and axes object
axes[0].hist(distributions[0].to("cpu").numpy(), label=str(0),
        bins=[0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9], density=True, histtype="step");
axes[1].hist(distributions[1].to("cpu").numpy(), label=str(1),
        bins=[0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9], density=True, histtype="step");
plt.legend()
plt.show()

In [ ]:
print(distributions[0][0:11].data)

In [ ]:
accuracy, loss, f1, f2, precision, recall, f1_fake, f2_fake, fake_precision, fake_recall = \
compute_accuracy(device, net, testloader, criterion, many_classes, "TEST", True)

print("Accuracy for the test data:",accuracy)
print("Loss for test data:",loss)
print("F1-score:",f1)
print("F2-score:",f2)
print("Precision:",precision)
print("Recall:",recall)
config.test_acc=accuracy
config.test_loss=loss
config.test_f1=f1
config.test_f2=f2
config.test_precis=precision
config.test_recall=recall

In [ ]:
for sequences, labels in testloader:
    sequences, labels = sequences.to(device), labels.to(device)
    outputs = net(sequences)
    _, predicted = torch.max(outputs.data, 1)
    for sequence, label, prediction, output in zip(sequences, labels, predicted, outputs):
        if label in [0,5,10,15]:
            if label==prediction:
                continue
            else:
                if prediction in [0,5,10,15]:
                    print(class_types[prediction], class_types[label],sequence, output)

In [ ]:
for sequences, labels in testloader:
    sequences, labels = sequences.to(device), labels.to(device)
    outputs = net(sequences)
    _, predicted = torch.max(outputs.data, 1)
    for sequence, label, prediction, output in zip(sequences, labels, predicted, outputs):
        if prediction==label:
            print(sequence,'\n\n',output,'\n\n', label,'\n\n', prediction, class_types[label.item()], 
                  class_types[prediction.item()])
            #print("TRUE:",class_types[prediction.item()])
            raise UserWarning('Exit Early')
        else:
            print("FALSE",sequence,'\n\n',output,'\n\n', label,'\n\n', prediction, class_types[label.item()],
                 class_types[prediction.item()])
            raise UserWarning('Exit Early')

In [ ]:
for sequences, labels in trainloader:
    for label in labels:
        if class_types[label.item()]=="TT":
            print("TT")

In [ ]:
test_tensor=torch.FloatTensor(np.array([[0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0],
                                [0,1,0,1,0,0,0,1,1,1,1,0,0,1,0,0,0,1,1,0,1,1,1,0,1,1,1,0,1,0,1,1,0],
                                [0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0],
                                [1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]]))
test_target=torch.FloatTensor(8) #G->A=8
test_tensor=test_tensor.reshape(1,1,4,33)
out_test=net(test_tensor)
print(out_test)

In [ ]:
predicted = torch.argmax(out_test, 1)
total =1
correct = (predicted == torch.argmax(test_target)).sum().item()
print(correct)

In [ ]:
test=np.zeros(6)
for luku in test:
    print(luku)

In [ ]:
print(torch.FloatTensor(np.array([[0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0],
                                [0,1,0,1,0,0,0,1,1,1,1,0,0,1,0,0,0,1,1,0,1,1,1,0,1,1,1,0,1,0,1,1,0],
                                [0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0],
                                [1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]])))

In [ ]:
net.eval()
with torch.no_grad():
    images, labels = iter(testloader).next()
    tests.plot_images(images[:5], n_rows=1)
    
    # Compute predictions
    images = images.to(device)
    y = net(images)

print('Ground truth labels: ', ' '.join('%10s' % classes[labels[j]] for j in range(5)))
print('Predictions:         ', ' '.join('%10s' % classes[j] for j in y.argmax(dim=1)))

In [ ]:
# Compute the accuracy on the test set
accuracy = compute_accuracy(net, testloader)
print('Accuracy of the network on the test images: %.3f' % accuracy)
assert accuracy > 0.85, "Poor accuracy {:.3f}".format(accuracy)
print('Success')